In [18]:
import findspark
findspark.init()
findspark.find()

'/home/hb/Downloads/spark-3.3.2-bin-hadoop3'

In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('bank').getOrCreate()

23/04/08 08:05:07 WARN Utils: Your hostname, hb resolves to a loopback address: 127.0.1.1; using 192.168.1.198 instead (on interface wlp3s0)
23/04/08 08:05:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/08 08:05:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
df = spark.read.csv("ML_hw_dataset.csv", inferSchema=True, header=True)

In [20]:
df.show(2)

+---+-----------+--------+-----------------+-------+-------+----+--------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|        education|default|housing|loan| contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+-----------+--------+-----------------+-------+-------+----+--------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 44|blue-collar| married|         basic.4y|unknown|    yes|  no|cellular|  aug|        thu|     210|       1|  999|       0|nonexistent|         1.4|        93.444|        -36.1|    4.963|     5228.1|  0|
| 53| technician| married|          unknown|     no|     no|  no|cellular|  nov|        fri|     138|       1|  999|       0|nonexistent|        -0.1|          93.2|        -42